In [1]:
import scib
import anndata as ad
import pandas as pd
import numpy as np
import os
from multiprocessing import Pool 
from scipy.io import mmread
from scipy.sparse import csr_matrix
import muon
import scarches as sca
import scanpy as sc
from scib_metrics.benchmark import Benchmarker
import scib_metrics
from typing import Optional

import warnings
warnings.filterwarnings("ignore")

/home/qukun/lyz32/miniconda3/envs/scarches/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/qukun/lyz32/miniconda3/envs/scarches/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/qukun/lyz32/miniconda3/envs/scarches/lib/python3.9

In [ ]:
# dt_list = ['3_brain_SNARE',
#          '4_brain_ISSAAC_seq',
#          '5_human_brain_10x',
#          '6_pbmc_granulocyte_sorted_10k',
#          '7_human_pbmc_10x',
#          '8_pbmc_unsorted_10k',
#          '9_pbmc_unsorted_3k',
#          '10_GSE201402_down',
#            '11_GSE194122',
#          '12_GSE156478_ctrl',
#          '12_GSE156478_stim']

dt_list = ['10_GSE201402_down']

In [4]:
for dataset in dt_list:
    latent = pd.DataFrame(np.load('../Integration/results/Vertical/RNA_ATAC/' + dataset + '/scMVP.npy'))
    latent.to_csv('../Integration/results/Vertical/RNA_ATAC/' + dataset + '/scMVP.csv', header = None, index=False)
    # latent = pd.read_csv('../Integration/results/Vertical/RNA_ATAC/' + dataset + '/scAI.csv', header = None)
    # latent.T.to_csv('../Integration/results/Vertical/RNA_ATAC/' + dataset + '/scAI.csv', header = None, index=False)

In [2]:
def read_RNA_ATAC(RNA_path,ATAC_path):
    # gene expression
    cell_names = pd.read_csv(RNA_path+'barcodes.tsv', sep = '\t', header=None, index_col=None)
    cell_names.columns =  ['cell_ids'] 
    cell_names['cell_ids'] = cell_names['cell_ids'].str.replace('.','-')
    X = csr_matrix(mmread(RNA_path+'matrix.mtx').T)
    gene_names = pd.read_csv(RNA_path+'features.tsv', sep = '\t',  header=None, index_col=None) 
    gene_names.columns =  ['gene_ids'] 
    adata_RNA = ad.AnnData(X, obs=pd.DataFrame(index=cell_names.cell_ids), var=pd.DataFrame(index = gene_names.gene_ids))
    adata_RNA.var_names_make_unique()
    # peak information
    cell_names = pd.read_csv(ATAC_path + 'barcodes.tsv', sep = '\t', header=None, index_col=None)
    cell_names.columns =  ['cell_ids'] 
    cell_names['cell_ids'] = cell_names['cell_ids'].str.replace('.','-')
    X = csr_matrix(mmread(ATAC_path + 'matrix.mtx').T)
    peak_name = pd.read_csv(ATAC_path + 'features.tsv', sep = '\t',header=None,index_col=None)
    peak_name.columns = ['peak_ids']
    adata_ATAC  = ad.AnnData(X, obs=pd.DataFrame(index=cell_names.cell_ids), var=pd.DataFrame(index = peak_name.peak_ids))
    return adata_RNA, adata_ATAC

In [6]:
def count_metrics(dataset):
    data_path = '../Integration/dataset/Vertical/RNA_ATAC/' + dataset
    adata_RNA, adata_ATAC = read_RNA_ATAC(data_path+"/RNA/",data_path+"/ATAC/")

    adata_RNA.layers["counts"] = adata_RNA.X.copy()
    sc.pp.normalize_total(adata_RNA)
    sc.pp.log1p(adata_RNA)
    sc.pp.highly_variable_genes(
        adata_RNA,
        flavor="seurat_v3",
        n_top_genes=4000,
        subset=False
    )
    adata_RNA = adata_RNA[:, adata_RNA.var.highly_variable].copy()

    adata_ATAC.layers['counts'] = adata_ATAC.X.copy()
    sc.pp.normalize_total(adata_ATAC, target_sum=1e4)
    sc.pp.log1p(adata_ATAC)
    adata_ATAC.layers['log-norm'] = adata_ATAC.X.copy()
    sc.pp.highly_variable_genes(adata_ATAC, n_top_genes=30000)
    adata_ATAC = adata_ATAC[:, adata_ATAC.var.highly_variable].copy()
    
    adata = sca.models.organize_multiome_anndatas(
        adatas = [[adata_RNA], [adata_ATAC]],    # a list of anndata objects per modality, RNA-seq always goes first
        layers = [['counts'], ['log-norm']], # if need to use data from .layers, if None use .X
    )

    metadata = pd.read_csv(data_path + "/meta_data.csv")
    metadata['celltype'].index = adata.obs_names
    adata.obs['cell_type'] = metadata['celltype'].astype('category')
    if np.where(adata.obs["cell_type"].isna())[0].shape[0]!=0:
        adata.obs["cell_type"] = adata.obs["cell_type"].cat.add_categories(['NaN'])
        adata.obs["cell_type"][np.where(adata.obs["cell_type"].isna())[0]] = 'NaN'
    adata.obs['batch'] = ['batch1'] * adata.shape[0]
    
    result = pd.DataFrame()
    metrics_list = []
    method_list = ['Multigrate','MultiVI','MOFA','SCOIT','MIRA','scMVP','scVAEIT','MOJITOO','DeepMaps','Schema','scAI']
    for method in method_list:
        if os.path.exists('../Integration/dataset/Vertical/RNA_ATAC/'+ dataset + '/'+ method + '.csv'):
            latent = pd.read_csv('../Integration/dataset/Vertical/RNA_ATAC/'+ dataset + '/'+ method + '.csv', header = None)
            latent.index = adata.obs_names
            adata.obsm[method] = latent
            sc.pp.neighbors(adata, use_rep=method)
            sc.tl.umap(adata)
            sc.tl.leiden(adata, key_added="cluster")
            scib.metrics.cluster_optimal_resolution(adata, cluster_key="cluster", label_key="cell_type")
            ari = scib.metrics.ari(adata, cluster_key="cluster", label_key="cell_type")
            iso_asw = scib.metrics.isolated_labels_asw(adata, label_key="cell_type", batch_key='batch', embed=method,  verbose = False)
            nmi = scib.metrics.nmi(adata, cluster_key="cluster", label_key="cell_type")
            clisi = scib.metrics.clisi_graph(adata, label_key="cell_type",use_rep=method, type_='embed')
            sht = scib.metrics.silhouette(adata, label_key="cell_type", embed=method, metric='euclidean', scale=True)
            metrics_list.append([ari, iso_asw, nmi, clisi, sht, method])
            
    con = mmread('../Integration/dataset/Vertical/RNA_ATAC/'+ dataset + '/'+ method + '_connectivities.mtx')
    dis = mmread('../Integration/dataset/Vertical/RNA_ATAC/'+ dataset + '/'+ method + '_distance.mtx')
    adata.uns['neighbors'] = {'connectivities_key': 'connectivities', 'distances_key': 'distances', 
                              'params': {'n_neighbors': 20, 'method': 'umap', 'random_state': 0, 
                                         'metric': 'euclidean'}}
    adata.uns['neighbors']['distance'] = csr_matrix(dis)
    adata.uns['neighbors']['connectivities'] = csr_matrix(con)
    adata.obsp['distance'] = csr_matrix(dis)
    adata.obsp['connectivities'] = csr_matrix(con)
    sc.tl.umap(adata, n_components=20)
    scib.metrics.cluster_optimal_resolution(adata, cluster_key="cluster", label_key="cell_type")
    ari = scib.metrics.ari(adata, cluster_key="cluster", label_key="cell_type")
    iso_asw = scib.metrics.isolated_labels_asw(adata, label_key="cell_type", batch_key='batch', embed=method,  verbose = False)
    nmi = scib.metrics.nmi(adata, cluster_key="cluster", label_key="cell_type")
    clisi = scib.metrics.clisi_graph(adata, label_key="cell_type",use_rep=method, type_='embed')
    sht = scib.metrics.silhouette(adata, label_key="cell_type", embed=method, metric='euclidean', scale=True)
    metrics_list.append([ari, iso_asw, nmi, clisi, sht, 'Seurat'])

    df = pd.DataFrame(metrics_list,columns = ['Isolated_Labels','method'])
    result = pd.merge(result,df, on='method', how='outer')
    result['Dataset'] = dataset
    result.to_csv(data_path + "/metrics_result.csv",index = False)
    print(dataset)

In [7]:
for dataset in dt_list:
    count_metrics(dataset)

         Falling back to preprocessing with `sc.pp.pca` and default params.
3_brain_SNARE
         Falling back to preprocessing with `sc.pp.pca` and default params.
4_brain_ISSAAC_seq
         Falling back to preprocessing with `sc.pp.pca` and default params.
5_human_brain_10x
         Falling back to preprocessing with `sc.pp.pca` and default params.
6_pbmc_granulocyte_sorted_10k
         Falling back to preprocessing with `sc.pp.pca` and default params.
7_human_pbmc_10x
         Falling back to preprocessing with `sc.pp.pca` and default params.
8_pbmc_unsorted_10k
         Falling back to preprocessing with `sc.pp.pca` and default params.
9_pbmc_unsorted_3k
         Falling back to preprocessing with `sc.pp.pca` and default params.
10_GSE201402_down
         Falling back to preprocessing with `sc.pp.pca` and default params.
12_GSE156478_ctrl
         Falling back to preprocessing with `sc.pp.pca` and default params.
12_GSE156478_stim
